# Script 1 para organização dos dados brutos dos exames
- Dado uma pasta de entrada e uma pasta de saída o programa varre a pasta de entrada convertendo todos os arquivos para o formato xlsx e os copia para a pasta de saída criando a mesma estrutura de diretórios da pasta de entrada.

In [ ]:
import os
import chardet
from openpyxl import Workbook

def detect_encoding(file_path):
    with open(file_path, 'rb') as file:
        detector = chardet.UniversalDetector()
        for line in file:
            detector.feed(line)
            if detector.done:
                break
        detector.close()
    return detector.result['encoding']

def convert_to_txt(input_file, output_file):
    # Detectando a codificação do arquivo
    encoding = detect_encoding(input_file)
    if encoding is None:
        encoding = 'utf-8'  # Caso a detecção de codificação não seja bem-sucedida, use UTF-8 como padrão

    # Lendo o arquivo em bytes
    with open(input_file, 'rb') as file:
        content_bytes = file.read()

    # Decodificando manualmente, ignorando erros
    content = content_bytes.decode(encoding, errors='ignore')

    # Escrevendo o conteúdo no arquivo de texto
    with open(output_file, 'w', encoding='utf-8') as txt_file:
        txt_file.write(content)

def txt_to_excel(input_folder_path, output_folder_path):
    # Criando a pasta de saída se não existir
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    # Iterando sobre os arquivos na pasta de entrada
    for root, dirs, files in os.walk(input_folder_path):
        for file_name in files:
            input_file_path = os.path.join(root, file_name)
            relative_path = os.path.relpath(root, input_folder_path)
            output_dir_path = os.path.join(output_folder_path, relative_path)
            if not os.path.exists(output_dir_path):
                    os.makedirs(output_dir_path)
            output_file_path = os.path.join(output_dir_path, f"{os.path.splitext(file_name)[0]}.xlsx")

            # Convertendo o arquivo de texto para Excel
            convert_to_txt(input_file_path, output_file_path)

            # Carregando o conteúdo do arquivo TXT
            with open(input_file_path, 'r', encoding='ISO-8859-1') as file:
                linhas = file.readlines()

            # Criando um novo arquivo Excel e escrevendo as linhas do arquivo TXT em colunas
            workbook = Workbook()
            sheet = workbook.active

            for linha in linhas:
                # Divide a linha com base na tabulação e adiciona as partes como colunas
                colunas = linha.strip().split('\t')
                sheet.append(colunas)

            # Salvando o arquivo Excel
            workbook.save(output_file_path)

            print(f"Conversão de txt para excel concluída para {file_name}. Arquivo de saída salvo em {output_file_path}")

if __name__ == "__main__":
    input_folder_path = input("Digite o caminho da pasta de entrada: ")
    output_folder_path = input("Digite o caminho da pasta de saída para os arquivos Excel: ")

    # Chamando a função para converter de TXT para Excel
    txt_to_excel(input_folder_path, output_folder_path)

    

    print("Conversão de TXT para Excel concluída.")
